**Cross validation**

Steps:
- split
  - k fold
  - leave one out
  - leave k out
- eval
- aggragate

Dependency:
- Model class
  - fit
  - predict
- Metrics class (can be part of model class)
  - eval


In [65]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [66]:
import sklearn.datasets as toy_data
import numpy as np
from abc import ABC, abstractmethod

X, y = toy_data.load_diabetes(return_X_y=True)
y = y.reshape(-1, 1)

class BaseCv(ABC):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    @abstractmethod
    def __len__(self):
        pass

    @abstractmethod
    def __getitem__(self, idx: int):
        pass

class kFoldCv(BaseCv):
    def __init__(self, X, y, k):    
        super().__init__(X, y)  
        self.k = k   

    def __len__(self):
        return self.k

    def __getitem__(self, idx):
        n = len(self.X)
        fold_len = n // self.k
        eval_idx = list(range(fold_len * idx, fold_len * (idx+1)))
        train_idx = [j for j in range(n) if j not in eval_idx]

        return self.X[train_idx], self.y[train_idx], self.X[eval_idx], self.y[eval_idx]

KFCV = kFoldCv(X, y, 5)

class LinReg:
    def __init__(self, bias=True):
        self.bias = bias

    def fit(self, X, y):
        if self.bias:
            ones = np.ones_like(y)
            X_u = np.concatenate([X, ones], axis=1)

        self.weights = np.linalg.inv(X_u.T @ X_u) @ (X_u.T) @ (y)

    def predict(self, X):
        if self.bias:
            ones = np.ones(shape=(len(X), 1))
            return np.concatenate([X, ones], axis=1) @ self.weights
        else:
            return X @ self.weights

model = LinReg()

class RMSE:
    def __init__(self):
        pass

    def __call__(self, y_pred, y_true):
        return np.sqrt(np.mean((y_pred - y_true) ** 2))
    
score = RMSE()


def run_cross_validation(cv_iter, model, score, name):
    scores = []
    agg_y_pred = []
    agg_y_truth = []
    for i, data in enumerate(cv_iter):
        X_t, y_t, X_e, y_e = data

        mdl = model.fit(X_t, y_t)
        pred = model.predict(X_e)

        scores.append(score(pred, y_e))
        agg_y_pred.append(pred)
        agg_y_truth.append(y_e)


    agg_y_pred = np.vstack(agg_y_pred)
    agg_y_truth = np.vstack(agg_y_truth)
    print('======')
    if name is not None:
        print(f'Cross validation run: {name}')

    print(f'Cross validation score mean {np.mean(scores)}, std {np.std(scores)}')
    print(f'Aggregated cross validation score: {score(agg_y_pred, agg_y_truth)}')
    print(f'Cross validation scores {scores}')
    print('======')

run_cross_validation(KFCV, model, score, 'my CV')


Cross validation run: my CV
Cross validation score mean 54.868924811339184, std 1.4108715713851354
Aggregated cross validation score: 54.88706103029507
Cross validation scores [52.77838976271778, 54.72325609766083, 57.1663145725575, 55.21340861259678, 54.46325501116304]


In [63]:
from sklearn.linear_model import LinearRegression

model_sklearn = LinearRegression()
run_cross_validation(KFCV, model_sklearn, score, '[5-fold][rmse] my CV over sklearn linear reg')

kfcv_50 = kFoldCv(X, y, k=50)
run_cross_validation(kfcv_50, model_sklearn, score, '[50-fold][rmse] my CV over sklearn linear reg')
run_cross_validation(kfcv_50, model, score, '[50-fold][rmse] my CV over my linear reg')


Cross validation run: [5-fold][rmse] my CV over sklearn linear reg
Cross validation score mean 54.86892481133924, std 1.4108715713851592
Aggregated cross validation score: 54.88706103029512
Cross validation scores [52.778389762717744, 54.72325609766089, 57.1663145725575, 55.21340861259704, 54.463255011163014]
Cross validation run: [50-fold][rmse] my CV over sklearn linear reg
Cross validation score mean 54.372820768181526, std 13.944684289967404
Aggregated cross validation score: 56.13250268993585
Cross validation scores [41.74456653410065, 47.6145416703316, 43.21290827374432, 56.63326025368588, 59.39173131349545, 51.474957505660896, 49.01661759484359, 70.111923585523, 36.13425322560359, 71.13826737804666, 37.61274783657284, 58.63464209277009, 70.97570568188247, 54.86958562960712, 60.8442993099554, 48.81995550997532, 44.561722231901044, 59.20342407101822, 54.915831910624775, 63.42225902708669, 41.07095165379506, 41.46503084916007, 43.15559966981412, 66.6087081415594, 51.506838100770764

In [68]:
class LooCV(BaseCv):
    def __init__(self, X, y):
        super().__init__(X, y)
        self.n = len(X)

    def __len__(self):
        return self.n
    
    def __getitem__(self, idx: int):
        train_idx = [j for j in range(self.n) if j != idx]
        return self.X[train_idx], self.y[train_idx], self.X[idx].reshape(1,-1), self.y[idx].reshape(1,-1)
    
loocv = LooCV(X, y)
allFoldCv = kFoldCv(X, y, len(X))

run_cross_validation(loocv, model, score, '[LOO][rmse] my CV over my linear reg')     
run_cross_validation(loocv, model_sklearn, score, '[LOO][rmse] my CV over sklearn linear reg')
run_cross_validation(allFoldCv, model, score, f'[len(X)({len(X)})-fold][rmse] my CV over my linear reg')     
run_cross_validation(allFoldCv, model_sklearn, score, f'[len(X)({len(X)})-fold][rmse] my CV over sklearn linear reg')

Cross validation run: [LOO][rmse] my CV over my linear reg
Cross validation score mean 44.35572304615822, std 32.16088742637479
Aggregated cross validation score: 54.78825464458093
Cross validation scores [56.10657450011209, 7.087310247770773, 36.74806962553029, 39.8516644145152, 6.623425552774393, 9.548296877562166, 65.17856258954289, 57.75463668352667, 49.89561002567498, 98.80255470000213, 4.084790766133153, 27.72016556619073, 64.41234843394403, 20.54411850307403, 15.230637992014834, 6.502336816010768, 46.79548828792841, 39.45233794735083, 51.48262599221417, 44.39151670467032, 53.24323662047183, 37.67790984635367, 46.09333185907684, 8.389594705010097, 18.79446534595624, 55.274525333620346, 40.999668301301654, 96.01035334569352, 1.9995959280990405, 102.44495981915725, 30.14410992129436, 10.625566326389048, 85.3236011499063, 26.979406310616824, 13.592107265350052, 15.123933399856085, 58.20195396600647, 119.91487281820721, 11.41449377495573, 47.93775445498855, 54.72188269575, 19.6809335